In [1]:
# ----------------------패키지 임포트, API 인증----------------------
!pip install pymysql
!pip install selenium
!pip install konlpy
!pip install google-cloud-vision
import tweepy
from tweepy import OAuthHandler, API
import pandas as pd
import datetime
import csv
import pandas
import re
import konlpy
from konlpy.tag import Okt
from collections import Counter
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import time
import webbrowser
import os 
import xlrd
import pymysql
from selenium.webdriver.chrome.options import Options


# API key 입력
CONSUMER_KEY = ''     
CONSUMER_SECRET = '' 
ACCESS_TOKEN = '' 
ACCESS_TOKEN_SECRET = ''    

# API 접근
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit = True) # 크롤링 횟수 제한 풀기


#--------------------------------05/23 추가-----------------------------------
#DB 연동
data_words_db = pymysql.connect(
    user='',
    passwd='',
    host='database-1.csxshuumxmig.ap-northeast-2.rds.amazonaws.com',
    db='project',
    charset='utf8mb4',
    use_unicode=True
)
print('접속되었습니다')

     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 911kB 7.8MB/s 
     |████████████████████████████████| 19.4MB 79.2MB/s 
     |████████████████████████████████| 450kB 47.1MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 471kB 8.2MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 


접속되었습니다


In [2]:
#DB 접속
cursor = data_words_db.cursor()
sql = "SELECT * FROM data_set;"
cursor.execute(sql)
result = cursor.fetchall()
df1 = pd.DataFrame(result)
df1.columns = ['number','main_set','sub_set']
#df1 = 기존의 단어셋

#--------------------------------05/23 추가-----------------------------------
df1_1=pd.DataFrame()
# 빈 데이터 프레임 생성
df1_sum = pd.concat([df1['main_set'],df1['sub_set']])
#df1_sum에 기존의 단어셋인 main과 sub를 결합
df1_sum=df1_sum.reset_index(drop=True) 
#인덱스 초기화를 통해 컬럼의 row 번호를 재정렬하고 drop=true로 컬럼명 제거
print(df1_sum)
print('\n')

df_final=pd.concat([df1_1,df1_sum],ignore_index=True)
#df_final에 빈프레임(df1_1)과 기존 단어셋을 합친다(df1_sum)
df_final.columns=['node_name']
#df_final의 컬럼명을 node_name으로 정립
print(df_final)

# df1은 main_set과 sub_set이 있는 키워드 db


0                 대딸방
1                 룸싸롱
2               섹스파트너
3                  콜걸
4                 마사지
5                  안마
6                  출장
7                  후기
8                  셔츠
9                 노래방
10                파트너
11                 추천
12    Samanth72483370
13    SusanPr33929337
14    oartearz4yxWzDy
15          MouyKelly
16      babyfat_cheek
17      Lesli86294362
18    Dvy9YigoGtyi0SN
19    W3EPNyF7FpUX3fR
20    EsnnqZaLZqkgas2
21    KrWMc5llH3ifTK6
22    y7o2AqV9hUKJBOv
23                 유흥
24                 섹시
25                 69
26                 조건
27                 섹스
28                 대행
29                 오피
30                 강남
31                 만남
32                 애인
33                사이트
34                 강추
35         hyewon1000
36    UN6R7Z97NvGBM3v
37    AkpZxHUftv7fXG7
38    Corneli15440797
39        jujujip0702
40    0KAIyLxN1zy3zzS
41           hyoubeen
42    YvetteS11564413
43    kv8xqOAviEWHa0F
44          vvoottee4
45    6ea6

In [3]:
#DB 연동
cursor = data_words_db.cursor()
sql = "SELECT * FROM data_strange;"
cursor.execute(sql)
result = cursor.fetchall()
strange = pd.DataFrame(result,columns=['number','words'])
print(strange)
print(len(strange))

    number words
0        1  ㅁr사지
1        2  ㅁし사지
2        3  마人ㅏ지
3        4  ㅁⓡ사지
4        5   섹ㅅ|
5        6   섹ㅅl
6        7   섹ㅅi
7        8   섹ㅅ卜
8        9   섹ㅍr
9       10   섹ㅍし
10      11   섹ㅍⓡ
11      12   섹ㅍ卜
12      13   ㄷり행
13      14   ㄷH행
14      15   ㄷⓗ행
15      16   ㉢ㅐ행
16


In [4]:

df_F = pd.DataFrame({'a':[],'b':[]})

lst=[]
for i in strange['words']:
  lst.append(i)

lsts=[]
for j in df_final['node_name']:
  lsts.append(j)

df_F['a']=lst
df_F['b']=lsts[:len(strange)]
print(df_F)


       a                b
0   ㅁr사지              대딸방
1   ㅁし사지              룸싸롱
2   마人ㅏ지            섹스파트너
3   ㅁⓡ사지               콜걸
4    섹ㅅ|              마사지
5    섹ㅅl               안마
6    섹ㅅi               출장
7    섹ㅅ卜               후기
8    섹ㅍr               셔츠
9    섹ㅍし              노래방
10   섹ㅍⓡ              파트너
11   섹ㅍ卜               추천
12   ㄷり행  Samanth72483370
13   ㄷH행  SusanPr33929337
14   ㄷⓗ행  oartearz4yxWzDy
15   ㉢ㅐ행        MouyKelly


In [7]:
#--------------------------------05/23 추가-----------------------------------
addr = ('/Users/jeon-ilsin/Desktop/main_beta-main')
#df = pandas.read_csv('main_bad_words.csv', sep=',') 
# 추후 DB로 이동할 부분임
#--------------------------------05/23 추가-----------------------------------


def check_nan(value) :
    try :
        float(value)
        return True
    except ValueError :
        return False


num_needed = 20
tweet_list = [] # 트위터 내용
name_list = [] #닉네임
screen_name_list = [] #사용자 아이디
date_list = [] #날짜 저장
time_list = [] #트위터 시간 저장
url_list = [] #주소 저장
tw_id = [] #아이디 중복 확인을 위한 저장 공간
last_id = -1 # ID of last tweet seen
nouns_list = [] # 한글만 뽑은 문장에서 실제 단어인지 검사
ko_text = [] # 파악한 단어 한글만 뽑기
snstype = [] #snstype넣기
modify_time = [] # 검색시간
time_now = datetime.datetime.now() #검색시간 패키지 진값
media_list = [] #사진 링크
id_str_list = [] #id값 저장칸

tweet_list2 = [] # 트위터 내용
name_list2 = [] #닉네임
screen_name_list2 = [] #사용자 아이디
date_list2 = [] #날짜 저장
time_list2 = [] #트위터 시간 저장
url_list2 = [] #주소 저장
tw_id2 = [] #아이디 중복 확인을 위한 저장 공간
last_id = -1 # ID of last tweet seen
nouns_list2 = [] # 한글만 뽑은 문장에서 실제 단어인지 검사
ko_text2 = [] # 파악한 단어 한글만 뽑기
snstype2 = [] #snstype넣기
modify_time2 = [] # 검색시간
time_now = datetime.datetime.now() #검색시간 패키지 진값
media_list2 = [] 
search_word2 = []
id_str_list2 = [] #id값 저장칸


#--------------------------------05/23 추가-----------------------------------
main_keywords = df1['main_set'] #메인 키워드, CSV에서 Main 컬럼 호출
sub_keywords = df1['sub_set'] # 서브 컬럼 호출
main_keywords2=df_F['a']
sub_keywords2=df_F['b']
#--------------------------------05/23 추가-----------------------------------

count_main = 0 #메인 단어 리스트 카운트 값
count_sub = 0 # 서브 카운트 값
result_main = 0 #메인 값을 집어넣음
result_sub = 0 # 서브 단어를 집어넣음
search_word = [] #검색단어 저장
Done = 0 #main 종료를 위한 값
Exit = 0 #완전 종료를 위한 값
abc=0 #media 갯수 초기화 사용
test = 0 #num대입을 위한 값

print(df1['main_set'])


0                 대딸방
1                 룸싸롱
2               섹스파트너
3                  콜걸
4                 마사지
5                  안마
6                  출장
7                  후기
8                  셔츠
9                 노래방
10                파트너
11                 추천
12    Samanth72483370
13    SusanPr33929337
14    oartearz4yxWzDy
15          MouyKelly
16      babyfat_cheek
17      Lesli86294362
18    Dvy9YigoGtyi0SN
19    W3EPNyF7FpUX3fR
20    EsnnqZaLZqkgas2
21    KrWMc5llH3ifTK6
22    y7o2AqV9hUKJBOv
Name: main_set, dtype: object


In [8]:
#for i in main_keywords:  # 메인 키워드들의 단어를 하나씩 실행

    #if not check_nan(main_keywords[count_main]):  # nan수 검사 만약 nan이 없다면 시작


# 메인값 반복을 위한 while문 (메인 키워드의 갯수가 count_main 보다 높을 동안 계속 작동)
while count_main < len(main_keywords):
                        # 메인 단어를 result_main에저장
            result_main = main_keywords[count_main]
            count_sub = 0
            
            while count_sub < len(sub_keywords):
                    
                # 서브 단어를 result_sub에 저장
                result_sub = sub_keywords[count_sub]
                last_id = -1  # 다시 last id 초기화
                test = num_needed
                
                #while len(tweet_list) < num_needed:  # 트위터 검색
                while test > 0:  # 트위터 검색
                    try:
                        
                        new_tweets = api.search(q="{}+' '+{}".format(
                            result_main, result_sub), count=num_needed, max_id=str(last_id - 1), tweet_mode='extended')
                        
                    except tweepy.TweepError as e:
                        print("Error", e)
                        break
                    else:
                        if not new_tweets:
                            print("Searching is done, Words is :" + result_main +','+ result_sub)
                            break
                        else:
                            for tweet in new_tweets:
                                
                                id_str = tweet.id_str
                                id_str_list.append(id_str)
                                
                                screen_name = tweet.author.screen_name
                                user_name = tweet.author.name
                                tweet_text = tweet.full_text
                                tweet_list.append(tweet_text)
                                name_list.append(tweet.user.name)
                                screen_name_list.append(screen_name)

                                created_time = str(tweet.created_at)
                                time_split = created_time.split()
                                date_list.append(time_split[0])
                                time_list.append(time_split[1])
                                url_list.append('https://twitter.com/' + tweet.user.screen_name)
                                tw_id.append(tweet.id)
                                search_word.append(result_main + ' ' + result_sub)
                                snstype.append('Twiiter')
                                modify_time.append(time_now)
                                
                                nouns_list = (tweet.id) #공간 활용을 위한 허수값 넣기
                                ko_text = (tweet.id) #공간 활용을 위한 허수값 넣기
                             
                                
                                try:  
                                    media_list.append(tweet.extended_entities['media'][abc]['media_url'])
            
                                except:
                                    abc = "no_media_url"
                                    media_list.append(abc)
                         
                                test = test - 1
                                abc=0
                               
                    last_id = min(tw_id)  # 여기까지가 트위터 검색
                count_sub = count_sub +1     
                
            count_main = count_main + 1

KeyboardInterrupt: ignored

# ocr 검색

In [9]:
num_needed2 = 20
tweet_list2 = [] # 트위터 내용
name_list2 = [] #닉네임
screen_name_list2 = [] #사용자 아이디
date_list2 = [] #날짜 저장
time_list2 = [] #트위터 시간 저장
url_list2 = [] #주소 저장
tw_id2 = [] #아이디 중복 확인을 위한 저장 공간
last_id2 = -1 # ID of last tweet seen
nouns_list2 = [] # 한글만 뽑은 문장에서 실제 단어인지 검사
ko_text2 = [] # 파악한 단어 한글만 뽑기
snstype2 = [] #snstype넣기
modify_time2 = [] # 검색시간
time_now2 = datetime.datetime.now() #검색시간 패키지 진값
media_list2 = [] #사진 링크
id_str_list2 = [] #id값 저장칸

tweet_list2 = [] # 트위터 내용
name_list2 = [] #닉네임
screen_name_list2 = [] #사용자 아이디
date_list2 = [] #날짜 저장
time_list2 = [] #트위터 시간 저장
url_list2 = [] #주소 저장
tw_id2 = [] #아이디 중복 확인을 위한 저장 공간
last_id2 = -1 # ID of last tweet seen
nouns_list2 = [] # 한글만 뽑은 문장에서 실제 단어인지 검사
ko_text2 = [] # 파악한 단어 한글만 뽑기
snstype2 = [] #snstype넣기
modify_time2 = [] # 검색시간
time_now2 = datetime.datetime.now() #검색시간 패키지 진값
media_list2 = [] 
search_word2 = []
id_str_list2 = [] #id값 저장칸


#--------------------------------05/23 추가-----------------------------------
main_keywords2=df_F['a']
sub_keywords2=df_F['b']
#--------------------------------05/23 추가-----------------------------------

count_main2 = 0 #메인 단어 리스트 카운트 값
count_sub2 = 0 # 서브 카운트 값
result_main2 = 0 #메인 값을 집어넣음
result_sub2 = 0 # 서브 단어를 집어넣음
search_word2 = [] #검색단어 저장
Done2 = 0 #main 종료를 위한 값
Exit2 = 0 #완전 종료를 위한 값
abc2=0 #media 갯수 초기화 사용
test2 = 0 #num대입을 위한 값


In [ ]:
#for i in main_keywords:  # 메인 키워드들의 단어를 하나씩 실행

    #if not check_nan(main_keywords[count_main]):  # nan수 검사 만약 nan이 없다면 시작


# 메인값 반복을 위한 while문 (메인 키워드의 갯수가 count_main 보다 높을 동안 계속 작동)
while count_main2 < len(main_keywords2):
                        # 메인 단어를 result_main에저장
            result_main2 = main_keywords2[count_main2]
            count_sub2 = 0
            
            while count_sub2 < len(sub_keywords2):
                    
                # 서브 단어를 result_sub에 저장
                result_sub2 = sub_keywords2[count_sub2]
                last_id2 = -1  # 다시 last id 초기화
                test2 = num_needed2
                
                #while len(tweet_list) < num_needed:  # 트위터 검색
                while test2 > 0:  # 트위터 검색
                    try:
                        
                        new_tweets2 = api.search(q="{}+' '+{}".format(
                            result_main2, result_sub2), count2=num_needed2, max_id2=str(last_id2 - 1), tweet_mode='extended')
                        
                    except tweepy.TweepError as e:
                        print("Error", e)
                        break
                    else:
                        if not new_tweets2:
                            print("Searching is done, Words is :" + result_main2 +','+ result_sub2)
                            break
                        else:
                            for tweet in new_tweets2:
                                
                                
                                id_str2 = tweet.id_str2
                                id_str_list2.append(id_str2)
                                
                                screen_name2 = tweet.author.screen_name
                                user_name2 = tweet.author.name
                                tweet_text2 = tweet.full_text
                                tweet_list2.append(tweet_text2)
                                name_list2.append(tweet.user.name)
                                screen_name_list2.append(screen_name2)
                                

                              
                                created_time2 = str(tweet.created_at)
                                time_split2 = created_time.split()
                                date_list2.append(time_split[0])
                                time_list2.append(time_split[1])
                                url_list2.append('https://twitter.com/' + tweet.user.screen_name)
                                tw_id2.append(tweet.id)
                                search_word2.append(result_main + ' ' + result_sub)
                                snstype2.append('Twiiter')
                                modify_time2.append(time_now)
                                
                                nouns_list2 = (tweet.id) #공간 활용을 위한 허수값 넣기
                                ko_text2 = (tweet.id) #공간 활용을 위한 허수값 넣기
                                
                                try:  
                                    media_list2.append(tweet.extended_entities['media'][abc2]['media_url'])
            
                                except:
                                    abc2 = "no_media_url"
                                    media_list2.append(abc)
                         
                                test2 = test2 - 1
                                abc2=0
                               
                                test2 = test2 - 1                          
                               
                    last_id2 = min(tw_id2)  # 여기까지가 트위터 검색
                count_sub2 = count_sub2 +1     
                
            count_main2 = count_main2 + 1

In [ ]:
# RT 제거,데이터 중복제거,데이터 프레임 저장하기

df2 = pd.DataFrame({'아이디':screen_name_list,
                   '닉네임':name_list,
                   '내용':tweet_list,
                   '날짜':date_list,
                   '시간':time_list,
                   '주소':url_list,
                   '검색단어':search_word,
                   'nouns':nouns_list,
                   'ko_text':ko_text,
                   'SNS타입':snstype,
                   '현재 시간': modify_time,
                   '사진': media_list,
                   'id_str':id_str_list
                  })
df2 = df2[~df2.내용.str.contains("RT @")]
df2 = df2.drop_duplicates(['아이디'])
print(df2)


# df2는 서치한 트윗의 아이디, 닉네임, 내용, 날짜, 시간, 주소, 검색단어, nouns, ko_text 열이 있는 db


In [ ]:


df2_2 = pd.DataFrame({'아이디':screen_name_list2,
                   '닉네임':name_list2,
                   '내용':tweet_list2,
                   '날짜':date_list2,
                   '시간':time_list2,
                   '주소':url_list2,
                   '검색단어':search_word2,
                   'nouns':nouns_list2,
                   'ko_text':ko_text2,
                   'SNS타입':snstype2,
                   '현재 시간': modify_time2,
                   '사진': media_list2,
                   'id_str':id_str_list2
                  })
print(df2_2)


In [ ]:
#RT 제거
if not df2_2.empty:
  df2_2 = df2_2[~df2_2.내용.str.contains("RT @")]
else:
  df2_2 = df2_2

df2_2 = df2_2.drop_duplicates(['아이디'])
print(df2_2)

# df2는 서치한 트윗의 아이디, 닉네임, 내용, 날짜, 시간, 주소, 검색단어, nouns, ko_text 열이 있는 db



In [ ]:
df3 = pd.concat([df2,df2_2])
print(df3)

#df3는 기존 키워드랑 변형 키워드로 서치해서 탐지한 모든 트윗들

In [ ]:
#디비 twitter 데이터 입력
for i in range(0, len(df3)):
    sql = "INSERT INTO project.twitter(id,nickname,contents,date,time,url,search_word,ko_text,snstype,modify_time,id_str,media_url) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    values = df3['아이디'].iloc[i], df3['닉네임'].iloc[i], df3['내용'].iloc[i], df3['날짜'].iloc[i], df3['시간'].iloc[i], df3['주소'].iloc[i], df3['검색단어'].iloc[i], df3['ko_text'].iloc[i],df3['SNS타입'].iloc[i],time_now,df3['id_str'].iloc[i],df3['사진'].iloc[i]
    cursor.execute(sql,values)
    data_words_db.commit()


In [ ]:
#print(tweet_list)
def text_cleaning(text):
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글만 뽑는 정규표현식 (수정함)
  rtnono = re.compile('[RT]+')
  result = hangul.sub('',text)
  result = rtnono.sub('',text) # 공백으로 처리
  return result


In [ ]:
# df2는 서치한 트윗의 아이디, 닉네임, 내용, 날짜, 시간, 주소, 검색단어, nouns, ko_text 열이 있는 db

df2['ko_text'] = df2['내용'].apply(lambda x: text_cleaning(x))

#--------------------------------05/23 추가-----------------------------------
#korean_stopwords_path = addr + 'korean_stopwords.txt' 

sql = "SELECT * FROM data_stopword;"
cursor.execute(sql)
result = cursor.fetchall()

data_stopword = pd.DataFrame(result) # 데이터프레임 생성
data_stopword.columns=['number','stopword'] # 숫자 열과 stopword 열이 있음.
print(data_stopword)

stopwords = data_stopword['stopword']
#with open(korean_stopwords_path, encoding='UTF8') as f:
#  stopwords = f.readlines() # 불용어 파일 읽기


#--------------------------------05/23 추가-----------------------------------
stopwords = [x.strip() for x in stopwords] 

# (2/2) nouns 생성:의미있는 명사만 추출하는 함수 생성
def get_nouns(x):
  nouns_tagger = Okt()
  nouns = nouns_tagger.nouns(x)
  # 한 글자 키워드 제거
  nouns = [noun for noun in nouns if len(noun) > 1]
  # 불용어 제거
  nouns = [noun for noun in nouns if noun not in stopwords]
  return nouns

#ko_text에 이를 적용
#df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x)) # 생성한 ko_text파일 다시 전처리
print(df3.주소)
df3['nouns'] = df2['ko_text'].apply(lambda x: get_nouns(x))




In [ ]:
# DataFrame 에 들어있는 자료들을 확인하기 위해 상위 n개 자료를 보여줌

# 키워드 빈도 추출하기
# 말뭉치 추출
tweet_corpus = "".join(df2['ko_text'].tolist())
# 명사 키워드 추출
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(tweet_corpus)
count = Counter(nouns)
# 한 글자 키워드 제거
remove_char_counter = Counter({x:count[x] for x in count if len(x) > 1}) # Counter는 같은 단어가 몇개 들어있는지 알려줌

In [ ]:
# 키워드와 키워드 빈도 점수를 'df_howMany', 'df_size'라는 데이터 프레임의 피처로 생성
df_howMany = pd.DataFrame(remove_char_counter.items(),columns = ['node_old','nodesize_old'])

df_howMany_100up = df_howMany[df_howMany['nodesize_old'] >= 50]
#빈도수 50 이상만 출력
df_howMany_100up_sort = df_howMany_100up.sort_values(by='nodesize_old' ,ascending=False)
print(df_howMany_100up_sort)
#위에 명령어가 빈도수 제거인데 홀수개가 나올 수 있으므로 일단 제거



In [ ]:

# 리스트에 빈도수 100개 이상인 키워드 넣기.
'''top_lists = []
for nod in df_howMany_100up_sort:
  top_lists.append(nod)
print('\n')
print(top_lists)'''
# 여기서 도시 명을 제거할거임. 도시명은 불법광고 탐지에 도움이 안되므로.
# 예시: top_lists=['오피', '출장', '만남']

In [ ]:
# 예시: lists=['오피', '출장', '만남']

# print(result)
#--------------------------------05/23 추가-----------------------------------

sql = "SELECT * FROM data_country;"
cursor.execute(sql)
result = cursor.fetchall()


data_country = pd.DataFrame(result) # 데이터프레임 만들기. 숫자 열 + 도시 이름 열
data_country.columns=['number','country_name']
#print(data_country['country_name'])

#korean_city_names_path = (addr + '전국도시이름.txt')
#with open(korean_city_names_path,encoding='utf8') as f:
cities = data_country['country_name']
cities = [x.strip() for x in cities]
# cities = ['서울', '부산', '인천', '대구' 등 100~200개 써놓음]
#print(cities)

# 도시 이름 제거
for i in cities:
    not_cities = df_howMany_100up_sort[df_howMany_100up_sort.node_old != i]

#--------------------------------05/23 추가-----------------------------------
df4 = pd.DataFrame(not_cities)
df4.columns = ['node_old','nodesize_old']

print(df3)


In [ ]:
#수집된 단어 정보를 워드클라우드 추가 하기 위한 부분

sql = "SELECT * FROM words_cloud;"
cursor.execute(sql)
result = cursor.fetchall()
wd = pd.DataFrame(result)
wd.columns = ['node_old','nodesize_old']

print(wd)

new_words = df4['node_old']
new_words_size = df4['nodesize_old']


words_cloud = pd.concat([wd,df4], ignore_index=True)
words_cloud = words_cloud.groupby(by=['node_old'],as_index=False).sum()
words_cloud = words_cloud.sort_values(by=['nodesize_old'],axis=0, ascending=False)
print(words_cloud)

for i in range(0, len(words_cloud)):
    
    sql = "INSERT INTO `words_cloud`(`words`, `words_size`) VALUES(%s, %s) ON DUPLICATE KEY UPDATE `words_size`=VALUES(`words_size`);"
    values = words_cloud['node_old'].iloc[i],words_cloud['nodesize_old'].iloc[i]
    cursor.execute(sql,values)
    data_words_db.commit()                                                                                                          
                                                                                                                


In [ ]:
#단어 합치고 비교하는 곳
#print(df_final) # 기존 단어셋 한줄로 합친 거
#print('\n')
#print(df3) # 최종적으로 뽑은 단어셋
lst = []
for i in range(len(df4)):
  lst.append(df3.iloc[i][0])
#최종으로 뽑은 단어 1st 공간에 넣기

df5 = pd.DataFrame(lst)
#리스트 데이터 프레임으로 변형
print(df5)
print('\n')

#df_final = 기존 단어셋(main과 sub) 합친 것
real_final1 = df_final.drop_duplicates()
#real_final1 = 기존 단어셋에서 중복 제거를 한 것
real_final1=real_final1.reset_index(drop=True) # 인덱스 초기화해줌


#1sts에 중복제거된 기존 단어셋 저장
lsts=[]
for i in df_final['node_name']:
  lsts.append(i)

#old = 예전 단어셋
#new = 새로 수집된 단어셋
#final = 새로운 단어셋에서 오래된 것을 빼서 새로운 단어만 파악
old=set(lsts)
new=set(lst)
final=new-old


new_words_list=list(final)
#final을 리스트 형태에 삽입


df6=pd.DataFrame(new_words_list)
df6.columns=['node_name']
#new_words_list를 dateframe형태로 변환 후 컬럼 네임 부여
print(df6)

In [ ]:

#빈도수 확인된 단어를 짝수로 맞추고 쪼개는 부분
sql = "SELECT * FROM data_set;"
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['number','main_set','sub_set']

l = len(df6)
#print(l)
if  l % 2 != 0:
  df7 = df6.drop(df6.index[-1])
else:
  df7 = df6


test1 = df7['node_name'].sample(frac=0.5, random_state=2019)
test2 = df7['node_name'].drop(test1.index)


for i in range(0, test2.shape[0]):
    l=test2.iloc[i]
    o=test1.iloc[i]
    sql = "INSERT INTO project.data_set(main_set,sub_set) VALUES(%s,%s);"
    values = l,o
    cursor.execute(sql,values)
    data_words_db.commit()
    
data_words_db.close() 


In [ ]:
# csv 파일 생성하기

#final_csv = pd.DataFrame(node_df)
#final_csv.to_excel(addr + '최종키워드'+'.xlsx')
#pd.read_excel(addr + '최종키워드' + '.xlsx')

In [ ]:
# # 파일로 저장
# df.to_csv(addr + '최종파일'+'.csv', encoding='utf-8-sig')
# print(df)


# #--------------------------------05/23 추가-----------------------------------
# data_words_db.close()
# #--------------------------------05/23 추가-----------------------------------

# OCR 부분

In [ ]:
#DB 연동
data_words_db = pymysql.connect(
    user='py',
    passwd='1q2w3e4r!@#',
    host='database-1.csxshuumxmig.ap-northeast-2.rds.amazonaws.com',
    db='project',
    charset='utf8mb4',
    use_unicode=True
)

#DB 접속
cursor = data_words_db.cursor()
sql = "SELECT * FROM data_normal_test;"
cursor.execute(sql)
result = cursor.fetchall()
df_n = pd.DataFrame(result)
df_n = pd.DataFrame(result,columns = ['number','main_set','sub_set'])

In [ ]:
df_n_1=pd.DataFrame()
# 빈 데이터 프레임 생성
df_n_sum = pd.concat([df_n['main_set'],df_n['sub_set']])
#df1_sum에 기존의 단어셋인 main과 sub를 결합
df_n_sum=df_n_sum.reset_index(drop=True) 
#인덱스 초기화를 통해 컬럼의 row 번호를 재정렬하고 drop=true로 컬럼명 제거
#print(df_n_sum)
#print('\n')

df_final=pd.concat([df_n_1,df_n_sum],ignore_index=True)
#df_final에 빈프레임(df_n_1)과 기존 단어셋을 합친다(df_n_sum)
df_final.columns=['node_name']
#df_final의 컬럼명을 node_name으로 정립
#print(df_final)

# df_n은 main_set과 sub_set이 있는 키워드 db

In [ ]:
def check_nan(value) :
    try :
        float(value)
        return True
    except ValueError :
        return False


num_needed = 20
tweet_list = [] # 트위터 내용
name_list = [] #닉네임
screen_name_list = [] #사용자 아이디
date_list = [] #날짜 저장
time_list = [] #트위터 시간 저장
url_list = [] #주소 저장
tw_id = [] #아이디 중복 확인을 위한 저장 공간
last_id = -1 # ID of last tweet seen
nouns_list = [] # 한글만 뽑은 문장에서 실제 단어인지 검사
ko_text = [] # 파악한 단어 한글만 뽑기
snstype = [] #snstype넣기
modify_time = [] # 검색시간
time_now = datetime.datetime.now() #검색시간 패키지 진값
media_list = [] #사진 링크
id_str_list = [] #id값 저장칸

#-------------------------------------------------------------------
main_keywords = df_n['main_set'] 
sub_keywords = df_n['sub_set'] 
#-------------------------------------------------------------------

count_main = 0 #메인 단어 리스트 카운트 값
count_sub = 0 # 서브 카운트 값
result_main = 0 #메인 값을 집어넣음
result_sub = 0 # 서브 단어를 집어넣음
search_word = [] #검색단어 저장
Done = 0 #main 종료를 위한 값
Exit = 0 #완전 종료를 위한 값

test = 0

#print(df_n)

In [ ]:
# 메인값 반복을 위한 while문 (메인 키워드의 갯수가 count_main 보다 높을 동안 계속 작동)
while count_main < len(main_keywords):
                        # 메인 단어를 result_main에저장
            result_main = main_keywords[count_main]
            count_sub = 0
            
            while count_sub < len(sub_keywords):
                    
                # 서브 단어를 result_sub에 저장
                result_sub = sub_keywords[count_sub]
                last_id = -1  # 다시 last id 초기화
                test = num_needed
                
                #while len(tweet_list) < num_needed:  # 트위터 검색
                while test > 0:  # 트위터 검색
                    try:
                        
                        new_tweets = api.search(q="{}+' '+{}".format(
                            result_main, result_sub), count=num_needed, max_id=str(last_id - 1), tweet_mode='extended')
                        
                    except tweepy.TweepError as e:
                        print("Error", e)
                        break
                    else:
                        if not new_tweets:
                            print("Searching is done, Words is :" + result_main +','+ result_sub)
                            break
                        else:
                            for tweet in new_tweets:
                                
                                id_str = tweet.id_str
                                id_str_list.append(id_str)
                                
                                screen_name = tweet.author.screen_name
                                user_name = tweet.author.name
                                tweet_text = tweet.full_text
                                tweet_list.append(tweet_text)
                                name_list.append(tweet.user.name)
                                screen_name_list.append(screen_name)

                                created_time = str(tweet.created_at)
                                time_split = created_time.split()
                                date_list.append(time_split[0])
                                time_list.append(time_split[1])
                                url_list.append('https://twitter.com/' + tweet.user.screen_name)
                                tw_id.append(tweet.id)
                                search_word.append(result_main + ' ' + result_sub)
                                snstype.append('Twiiter')
                                modify_time.append(time_now)
                                
                                nouns_list = (tweet.id) #공간 활용을 위한 허수값 넣기
                                ko_text = (tweet.id) #공간 활용을 위한 허수값 넣기
                             
                                
                                try:  
                                    media_list.append(tweet.extended_entities['media'][abc]['media_url'])
            
                                except:
                                    abc = "no_media_url"
                                    media_list.append(abc)
                         
                                test = test - 1
                                abc=0
                               
                    last_id = min(tw_id)  # 여기까지가 트위터 검색
                count_sub = count_sub +1     
                
            count_main = count_main + 1

In [ ]:

df_n_1 = pd.DataFrame({'아이디':screen_name_list,
                   '닉네임':name_list,
                   '날짜':date_list,
                   '시간':time_list,
                   '주소':url_list,
                   '검색단어':search_word,
                   'SNS타입':snstype,
                   '현재 시간': modify_time,
                   '사진': media_list
                  })
df_n_1 = df_n_1.drop_duplicates(['아이디'])
#print(df_n_1)

## API

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'axiomatic-atlas-316112-69425609322a.json'
import urllib.request
import io
from google.cloud import vision
import ssl

#계정 정보 얻어옴
client = vision.ImageAnnotatorClient()

#사진 읽어오기 
lsts=[]
#def ocr(url):
client = vision.ImageAnnotatorClient()
for i in df_n_1['사진']:
  if i != 'no_media_url':
    with urllib.request.urlopen(i) as image_file: 
      content = image_file.read()
      lsts.append(content)
  else: lsts.append('None')
#print(len(lsts)) 
#print(len(df_n_1)) # 동일함. 잘 받아옴

In [ ]:
f_list=[]
for j in lsts:
  if j != 'None':
    image = vision.Image(content=j)
    response = client.text_detection(image=image) # 사진에서 텍스트 추출
    texts = response.text_annotations
    result = []
    for text in texts:
      result.append(text.description)
    f_list.append(result)
  else: f_list.append('none')

In [ ]:
eList=[]
for k in range(len(f_list)):
  eList.append(f_list[k][0])
#print(eList)

In [ ]:
url = df_n_1['주소']

In [ ]:
# columns=기존 트위터 DB 컬럼에 ocr text, ocr한 이미지 url 추가
import pandas as pd

url = list(url)
print(url)
print(len(url))

oxList=[]
for i in range(len(url)):
  oxList.append('X')

In [ ]:
df_ocr = pd.DataFrame({
                        '주소':url,
                        'ocr_text':eList,
                      })
#print(df_ocr)

In [ ]:
l_texts = list(df_ocr['ocr_text'])
#print(l_texts)

## 기존 단어셋 가져오기

In [ ]:
#DB 접속
cursor = data_words_db.cursor()
sql = "SELECT * FROM data_set;"
cursor.execute(sql)
result = cursor.fetchall()
df1 = pd.DataFrame(result)
df1.columns = ['number','main_set','sub_set']
#df1 = 기존의 단어셋

#-----------------------------------------------------------------
df1_1=pd.DataFrame()
# 빈 데이터 프레임 생성
df1_sum = pd.concat([df1['main_set'],df1['sub_set']])
#df1_sum에 기존의 단어셋인 main과 sub를 결합
df1_sum=df1_sum.reset_index(drop=True) 
#인덱스 초기화를 통해 컬럼의 row 번호를 재정렬하고 drop=true로 컬럼명 제거

df_final=pd.concat([df1_1,df1_sum],ignore_index=True)
#df_final에 빈프레임(df1_1)과 기존 단어셋을 합친다(df1_sum)
df_final.columns=['node_name']
#df_final의 컬럼명을 node_name으로 정립
#print(df_final)

# df1은 main_set과 sub_set이 있는 키워드 db
l_keys=[]
for i in df_final['node_name']:
  l_keys.append(i)
#print(l_keys)

In [ ]:
#print(l_texts)
#print(l_keys)

In [ ]:
real_final=[]
for i in l_texts:
  for j in l_keys:
    if j in i:
      real_final.append(i)

final_list=list(set(real_final))

In [ ]:
df_filter = pd.DataFrame(columns=['ocr_text'])
df_filter['ocr_text'] = final_list
#print(df_filter)

In [ ]:
df_sum = pd.concat([df_ocr,df_filter])
df_sum=df_sum.reset_index(drop=True) # 인덱스 초기화해줌


#print(df_filter)

In [ ]:
df_sum2 = pd.DataFrame(columns=['주소','ocr_text'])
for i in df_filter['ocr_text']:
  df_sum2 = df_sum.loc[df_sum['ocr_text']==i]

In [ ]:
df_sum2 = pd.DataFrame(columns=['주소','ocr_text'])
df_sum3 = pd.DataFrame(columns=['주소','ocr_text'])
df_sum4 = pd.DataFrame(columns=['주소','ocr_text'])
for i in df_filter['ocr_text']:
  df_sum2 = df_sum.loc[df_sum['ocr_text']==i]
  df_sum3 = df_sum3.append(df_sum2)
  df_sum3=df_sum3.reset_index(drop=True) # 인덱스 초기화해줌
  df_sum4 = df_sum3.iloc[::2,:]
  df_sum4=df_sum4.reset_index(drop=True)
#print(df_sum4)

In [ ]:
#print(df_n_1)
#print('------------')
#print(df_sum4)

In [ ]:
ocr_final = pd.merge(df_n_1, df_sum4, how='outer', on='주소')
ocr_final = ocr_final.where((pd.notnull(ocr_final)), None) # nan값 none으로 바꾸기
#ocr_final

## DB 연동

In [ ]:
#디비 data_ocr 데이터 입력
for i in range(0, len(df_n_1)):
    sql = "INSERT INTO project.data_ocr(id,nickname,date,time,url,search_word,snstype,modify_time,media_url,ocr_text) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    values = ocr_final['아이디'].iloc[i], ocr_final['닉네임'].iloc[i], ocr_final['날짜'].iloc[i], ocr_final['시간'].iloc[i], ocr_final['주소'].iloc[i], ocr_final['검색단어'].iloc[i],ocr_final['SNS타입'].iloc[i],time_now,ocr_final['사진'].iloc[i], ocr_final['ocr_text'].iloc[i]
    cursor.execute(sql,values)
    data_words_db.commit()